In [143]:
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

import keras
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

import numpy as np
import pandas as pd

import random
import os
import yaml
import pickle
from yaml import *

import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import cv2
from cv2 import cvtColor, GaussianBlur, resize

from PIL import Image

In [ ]:
#Checking the Tensorflow/Keras versions and seeing if GPU is running
import tensorflow as tf
print("Tensorflow Version: " + tf.__version__)
print('Keras Version: ' + keras.__version__)
print("Running on GPU:", tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#Limiting GPU Memory to reduce overloading
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

### Folder Structure

Below is the folder structure needed in the root of the notebook. The names with extensions (.png, .yaml, .ipynb) are files while the names without extensions are folders. The indentation is representative of the tree structure, if the strucutre is changed, make sure to change the code.

In [ ]:
"""
root
    -> data
        -> allInputImages.png
    -> info
        -> allInputYamls.yaml
    -> models
        -> checkpoints
        -> final
    -> test
        -> allValidationImages.png
        -> allValidationYamls.yaml
    -> Model Training.ipynb
"""

### Data Prep Functions

In [ ]:
#Function to create the lists of imagePaths and yamlData for the Machine Learning
def creatingLists():
    #Folders names data and info will need to be created in the root with the images fed into data and yamls into info
    imagePath = 'data'
    yamlPath = 'info'
    
    #List of paths
    imageList = os.listdir(imagePath)
    yamlList = os.listdir(yamlPath)
    
    imagePaths = []
    yamlData = []
    
    processNum = 0
    
    #Adding Image path to List
    for filename in sorted(imageList):
        imagePaths.append(os.path.join(imagePath, filename))
        processNum += 1
        print(filename + '  ' + str(processNum))
        
    processNum = 0
    
    #Parsing through each yaml file and adding its content to the list (will initially take some time)
    for filename in sorted(yamlList):
        with open(os.path.join(yamlPath, filename)) as file:
            yamlDataInput = yaml.safe_load(file)
            yamlData.append(yamlDataInput)
            processNum += 1
            print(filename + '   ' + str(processNum))
    
    return imagePaths, yamlData

In [ ]:
#Function to save the created lists to avoid parsing through Yaml files every time on load
def savingLists(listToSave, listName):
    try:
        #Saving Lists
        with open(listName + ".txt", 'wb') as f:
            pickle.dump(listToSave, f)
        print(listName + ' Saved!')
    except:
        print(listName + ' Failed to Save!')

#Function to open the saved lists
def openingLists(listName):
    listToOpen = []
    try:
        #Loading Lists
        with open(listName + ".txt", 'rb') as f:
            listToOpen = pickle.load(f)
        print(listName + ' Opened!')        
    except:
        print(listName + ' Failed to Open!')

    return listToOpen

In [ ]:
#Function to isolate a certain parameter in the Yaml data list and return a list of it
def isolateYAML(yamlData, parameter):
    yamlIsolate = []
    for yamls in yamlData:
        yamlIsolate.append(yamls[parameter])
    
    print(parameter + ' isolated')
    
    return yamlIsolate

In [ ]:
#Function to process the list of images (feed in the imagePathList) and make sure to enter whether to invert the images or not
def processImages(imageList, inverseSize):
    
    ImageArray = []
    
    index = 0
    for image in imageList:
        src = cv2.imread(image)
        src = cv2.cvtColor(src, cv2.COLOR_RGB2YUV)
        src = cv2.GaussianBlur(src, (3,3), 0)
        
        if inverseSize is False:
            imageAppend = cv2.resize(src, (200,150))
        else:
            imageAppend = cv2.resize(src, (150,200))
        
        ImageArray.append(imageAppend)
        index += 1
        print(str(index) + "  Images Processed")
    
    
    ImageArray = np.array(ImageArray)
    
    print("Shape of Images: ", ImageArray.shape)
    
    return ImageArray

In [ ]:
#Function to split the dataset into train, test sets (Consider adding a cross-validation set)
def trainTestSplit(imagePaths, yamlData, splitPercent):
    trainX, testX, trainY, testY = train_test_split(imagePaths, yamlData, test_size = splitPercent)
    print("Training Data: %d\nValidation Data: %d" % (len(trainX), len(testX)))
    return trainX, testX, trainY, testY

In [ ]:
#Single Function to see how long the model training process takes, to use look at comment below
    # start the training process with a variable startTime = time.perf_counter()
    # end the training with runTime(startTime)
def runTime(startTime):
    currentTime = time.perf_counter()
    hours = (int)((currentTime - startTime) / 3600)
    minutes = (int)(((currentTime - startTime) - (3600 * hours)) / 60)
    seconds = (int)((currentTime - startTime) - (3600 * hours) - (60 * minutes))
    
    print("\n__Time__", "\nHours:", hours, "\nMinutes:", minutes, "\nSeconds:", seconds)

### Model Definitions
The functions below are different models, It is recommened to make modifications to a duplicate of a model instead of changes directly to the model.

In [1]:
def nvidiaBaseModel():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (200,150,3), activation='elu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Flatten())
    
    model.add(Dense(1164, activation='elu'))
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_Dropout():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (200,150,3), activation='elu'))
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    
    model.add(Dense(1164, activation='elu'))
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def cifarBaseModel_DropoutMaxpool():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 150, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def cifarBaseModel_DropoutMaxpoolElu():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 150, 3)))
    model.add(Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='elu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_DenseSoftmax():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (200,150,3), activation='elu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Flatten())
    
    model.add(Dense(1164, activation='softmax'))
    model.add(Dense(100, activation='softmax'))
    model.add(Dense(50, activation='softmax'))
    model.add(Dense(10, activation='softmax'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_DenseSoftmaxDropout():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (640,480,3), activation='elu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    modelel.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    
    model.add(Dense(1164, activation='softmax'))
    model.add(Dense(100, activation='softmax'))
    model.add(Dense(50, activation='softmax'))
    model.add(Dense(10, activation='softmax'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_DenseRelu():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (200,150,3), activation='elu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Flatten())
    
    model.add(Dense(1164, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_DenseReluDropout():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (200,150,3), activation='elu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    
    model.add(Dense(1164, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_FullRelu():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (640,480,3), activation='relu'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='relu'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu'))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1164, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def nvidiaBaseModel_FullSigmoid():
    model = Sequential()
    
    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape= (640,480,3), activation='sigmoid'))
    
    model.add(Conv2D(24, (5,5), strides=(2,2), activation='sigmoid'))
    
    model.add(Conv2D(36, (5,5), strides=(2,2), activation='sigmoid'))
    
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='sigmoid'))
    
    model.add(Conv2D(64, (3,3), strides=(2,2), activation='sigmoid'))
    
    model.add(Flatten())
    
    model.add(Dense(1164, activation='sigmoid'))
    model.add(Dense(100, activation='sigmoid'))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(10, activation='sigmoid'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=1e-5)
    model.compile(loss='mse', optimizer=optimizer)
    
    print(model)
    return model

In [ ]:
def baseTransferModel():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(200,150,3)))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Conv2D(32, (5, 5)))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (5, 5)))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(3, activation=None))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9, clipnorm=1)
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])
    
    return model

### Model Parameters

In [ ]:
# Be sure to have imagePaths and yamlData folders made in root
imagePathsName = "imagePaths"
yamlDataName = "yamlData"

#Creating Lists of Images and Yaml Data (Run only once the first time yamlData is parsed, comment out afterwards)
imagePaths, yamlData = creatingLists()

#Saving the Images and Yaml Lists (Run only once the first time yamlData is parsed, comment out afterwards)
savingLists(imagePaths, imagePathsName)
savingLists(yamlData, yamlDataName)

In [ ]:
#Opening Image and Yaml Data Lists (Instead of creating the lists everytime)
imagePaths = openingLists(imagePathsName)
yamlData = openingLists(yamlDataName)

In [ ]:
#Isolate Yaml Properties
yamlIsolateSteerAngle = isolateYAML(yamlData, 'steering_angle')
yamlIsolateSteerCommand = isolateYAML(yamlData, 'steering_command')

In [ ]:
#Proccessing the images into the ImageData list (a list of image arrays)
ImageData = processImages(imagePaths, False)

In [ ]:
#Splitting data into individual sets (Change the second input parameter into which list to compare the images to, the thrid parameter is the split%)
trainX, testX, trainY, testY = trainTestSplit(ImageData, yamlIsolateSteerAngle, 0.2)

### Model Training

In [ ]:
# Start Time definition
startTime = time.perf_counter()

#Modify these when running different models
modelName = 'nvidiaBaseModel_Dropout'
model = nvidiaBaseModel_Dropout()

#The saving directory for the models (make sure to create a models folder with a checkpoints and final folder inside of them)
modelOutputDirCheckpoint = os.path.join(os.getcwd(), 'models', 'checkpoints')
modelOutputDirFinal = os.path.join(os.getcwd(), 'models', 'final')

#Converting sets to arrays
trainX = np.asarray(trainX)
testX = np.asarray(testX)
trainY = np.asarray(trainY)
testY = np.asarray(testY)
print('Model Done')

#Model Variable definition
batchSize = 64
numEpochs = 10

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
itTrain = datagen.flow(trainX, trainY, batch_size=batchSize)
steps = int(trainX.shape[0] / batchSize)

#Checkpoints (incase crash while training)
checkpointCallback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(modelOutputDirCheckpoint, modelName + '_NavigationCheck.h5'), verbose=1, save_best_only=True)
print('Prerequisites done')

#The model training
history = model.fit_generator(itTrain,
                              steps_per_epoch=steps,
                              epochs=numEpochs,
                              validation_data=(testX,testY),
                              verbose=1,
                              callbacks=[checkpointCallback])
print('history done')

#Saving the final model
model.save(os.path.join(modelOutputDirFinal, modelName + '_NavigationFinal_' + 'BatchSize:' + str(batchSize) + '_Epochs:' + str(numEpochs) + '_.h5'))
print('model saved')

#See how long everything took to run
runTime(startTime)

In [ ]:
#Some basic metrics on the model
plt.plot(history.history['loss'], color='blue')
plt.plot(history.history['val_loss'], color='red')
plt.legend(['training loss', 'validation loss'])

### Model Validation

In [147]:
#Function to predict the outut for a model based on input image (image is already processed as numpy array)
def computeParams(image):
    model = load_model(os.path.join(modelOutputDirFinal, modelName + '_NavigationFinal_' + 'BatchSize:' + str(batchSize) + '_Epochs:' + str(numEpochs) + '_.h5'))
    params = model.predict(image)
    return params

In [ ]:
#Creating Validation lists
testPath = os.listdir('test')
imageTestPathVal = []
yamlTestPathVal = []

numImages = 100

indexing = 0;
for file in sorted(testPath):
    if indexing < (numImages*2):
        if file.endswith('.png'):
            imageTestPathVal.append(os.path.join('test', file))
        if file.endswith('.yaml'):
            yamlTestPathVal.append(os.path.join('test', file))
        indexing += 1
    else:
        break

print('Number of Images Loaded: ' + str(len(imageTestPathVal)))
print('Number of Yamls Loaded: ' + str(len(yamlTestPathVal)))

In [ ]:
#Preping the Validation images
imageTestVal = processImages(imageTestPathVal, True)

In [ ]:
#Creating lists on actual angles based on yaml files and predicted angles based on model for the validation set
index = 0

modelOutput = []
actualAngle = []

for index1 in range(len(imageTest)):
    index2 = computeParams(imageTestVal)[[index1]]
    for index3 in index2:
        for index4 in index3:
            print(index4)
            modelOutput.append(index4)

for filename in yamlTestPathVal:
    with open(filename) as file:
        yamlDataInput = yaml.safe_load(file)
        print(yamlDataInput['steering_angle'])
        actualAngle.append(yamlDataInput["steering_angle"])

In [ ]:
#Graphing the Validation set Real vs Predicted
plt.plot(modelOutput, color = 'orange')
plt.plot(actualAngle, color = 'green')

plt.legend(['Output', 'Real'])

### Support
Most of the important code is commented with its purpose and functionality. Feel free to make changes and run the model. 

To create the data, read through the instruction on the main github page README to get the docker enviornment running. Then after the **source devel/setup.bash** step, cd into src/road_dataset_generation/scripts (or wherever the generate_dataset.py file is), then run the **./generate_dataset.py** command. If you want a fresh dataset (clear old data), then type y, else n.

Check out the link below for some additional information:
https://towardsdatascience.com/deeppicar-part-5-lane-following-via-deep-learning-d93acdce6110

If there are any more questions or comments, feel free to contact me via email at bishneet.singh@uwaterloo.ca or n2deshmu@uwaterloo.ca or via teams.